In [2]:
from pydantic_ai import Agent, RunContext
from pydantic_ai.common_tools.tavily import tavily_search_tool
from pydantic_ai.messages import ModelMessage
from dotenv import load_dotenv
import os
from pydantic import Field
from deep_research import Deep_research_engine
from pydantic_ai.models.gemini import GeminiModel
from pydantic_ai.providers.google_gla import GoogleGLAProvider
from dataclasses import dataclass
from presentation_generator import Presentation_gen
from typing import Optional
from spire.doc import Document,FileFormat
from spire.doc.common import *
import requests
from PIL import Image
from io import BytesIO
import tempfile



In [13]:
load_dotenv()
tavily_key=os.getenv('tavily_key')
google_api_key=os.getenv('google_api_key')
pse=os.getenv('pse')

llm=GeminiModel('gemini-2.0-flash', provider=GoogleGLAProvider(api_key=google_api_key))

deepsearch=Deep_research_engine()
presentation=Presentation_gen()

@dataclass
class Deps:
    deep_search_results:list[dict]
    quick_search_results:list[str]
    # presentation_slides:dict



async def deep_research_agent(ctx:RunContext[Deps], query:str):
    """
    This function is used to do a deep research on the web for information on a complex query, generates a report or a paper.
    Args:
        query (str): The query to search for
    Returns:
        str: The result of the search
    """
    res=await deepsearch.chat(query)
    ctx.deps.deep_search_results.append(res)
    return str(res)

quick_search_agent=Agent(llm,tools=[tavily_search_tool(tavily_key)])
async def quick_research_agent(ctx: RunContext[Deps], query:str):
    """
    This function is used to do a quick search on the web for information on a given query.
    Args:
        query (str): The query to search for
    Returns:
        str: The result of the search
    """
    res=await quick_search_agent.run(query)
    ctx.deps.quick_search_results.append(res.data)
    return str(res.data)


def google_image_search(query:str):
  """Search for images using Google Custom Search API
  args: query
  return: image url
  """
  # Define the API endpoint for Google Custom Search
  url = "https://www.googleapis.com/customsearch/v1"

  params = {
      "q": query,
      "cx": pse,
      "key": google_api_key,
      "searchType": "image",  # Search for images
      "num": 1  # Number of results to fetch
  }

  # Make the request to the Google Custom Search API
  response = requests.get(url, params=params)
  data = response.json()

  # Check if the response contains image results
  if 'items' in data:
      # Extract the first image result
      image_url = data['items'][0]['link']
      return image_url
  


async def research_editor_tool(ctx: RunContext[Deps], query:str):
    """
    Use this tool to edit the deep search result to make it more accurate following the query's instructions.
    This tool can modify paragraphs, image_url or table. For image_url, you need to give the query to search for the image.
    Args:
        query (str): The query containing instructions for editing the deep search result
    Returns:
        str: The edited and improved deep search result
    """
    @dataclass
    class edit_route:
        page_number:int = Field(description='the page number to edit')
        paragraph_number:Optional[int] = Field(default_factory=None, description='the number of the paragraph to edit, if the paragraph is not needed to be edited, return None')
        route: str = Field(description='the route to the content to edit, either paragraphs, image_url, edit_table or add_table')

    paper_dict={}
    for i in range(len(ctx.deps.deep_search_results)):
        paper_dict[f'page_{i+1}']={'title':ctx.deps.deep_search_results[i].get('title'),
                                   'image_url':ctx.deps.deep_search_results[i].get('image_url') if ctx.deps.deep_search_results[i].get('image_url') else 'None',
                                   'paragraphs_title':{num:paragraph.get('title') for num,paragraph in enumerate(ctx.deps.deep_search_results[i].get('paragraphs'))}, 
                                   'table':ctx.deps.deep_search_results[i].get('table') if ctx.deps.deep_search_results[i].get('table') else 'None',
                                   'references':ctx.deps.deep_search_results[i].get('references')}
    
    route_agent=Agent(llm,result_type=edit_route, system_prompt="you decide the route to the content to edit based on the query's instructions and the paper_dict, either paragraphs, image_url or table")
    route=await route_agent.run(f'query:{query}, paper_dict:{paper_dict}')
    contents=ctx.deps.deep_search_results[route.data.page_number-1]
 
    
    @dataclass
    class Research_edits:
        edits:str = Field(description='the edits')
    editor_agent=Agent(llm,tools=[google_image_search],result_type=Research_edits, system_prompt="you are an editor, you are given a query, some content to edit, and maybe a quick search result (optional), you need to edit the deep search result to make it more accurate following the query's instructions, return only the edited content, no comments")
    if route.data.route=='paragraphs':
        content=contents.get('paragraphs')[route.data.paragraph_number]['content']
        res=await editor_agent.run(f'query:{query}, content:{content}, quick_search_results:{ctx.deps.quick_search_results if ctx.deps.quick_search_results else "None"}')
        ctx.deps.deep_search_results[route.data.page_number-1]['paragraphs'][route.data.paragraph_number]['content']=res.data.edits
    if route.data.route=='image_url':
        content=contents.get('image_url')
        res=await editor_agent.run(f'query:{query}, content:{content}, quick_search_results:{ctx.deps.quick_search_results if ctx.deps.quick_search_results else "None"}')
        ctx.deps.deep_search_results[route.data.page_number-1]['image_url']=res.data.edits
    if route.data.route=='edit_table':
        content=contents.get('table')
        res=await editor_agent.run(f'query:{query}, content:{content}, quick_search_results:{ctx.deps.quick_search_results if ctx.deps.quick_search_results else "None"}')
        ctx.deps.deep_search_results[route.data.page_number-1]['table']=res.data.edits
    if route.data.route=='add_table':
 
        res=await editor_agent.run(f'query:{query}, quick_search_results:{ctx.deps.quick_search_results if ctx.deps.quick_search_results else "None"}, research_results:{ctx.deps.deep_search_results[route.data.page_number-1]}')
        ctx.deps.deep_search_results[route.data.page_number-1]['table']=res.data.edits
    
    return str(ctx.deps.deep_search_results)


# async def presentation_agent(
#     ctx: RunContext[Deps],
#     style: Optional[str] = "professional",
#     instruction: Optional[str] = 'None'
# ) -> dict:
#     """
#     Generate a presentation based on the deep search result.

#     Args:
#         style: Presentation style (default: "professional")
#         instruction: Additional instructions for presentation generation (default: None)

#     Returns:
#         dict: The presentation slides
#     """
#     search_content = (ctx.deps.edited_deep_search_result 
#                      if ctx.deps.edited_deep_search_result 
#                      else ctx.deps.deep_search_results[-1])
    
#     res = await presentation.chat(search_content, presentation_style=style, instruction=instruction)
#     ctx.deps.presentation_slides = res
#     return 'presentation generated'

# @dataclass
# class page_edit:
#     edits: str = Field(description='the edited page in html format')

# page_editor=Agent(llm, result_type=page_edit, system_prompt='you are a page editor, edit the mentionned page(s) based on the instructions.')
# async def page_editor_tool(ctx: RunContext[Deps], page_number: int, instructions: str):
#     """
#     This function is used to edit the mentionned page(s) based on the instructions.
#     Args:
#         page_number (int): The number of the page to edit
#         instructions (str): The instructions for the page edits
#     Returns:
#         str: The edited page
#     """
#     result=await page_editor.run(f'page:{ctx.deps.presentation_slides[f"page_{page_number}"]}, instructions:{instructions}')
#     ctx.deps.presentation_slides[f"page_{page_number}"]=result.data.edits
#     return f'page{page_number} edited'


In [14]:
@dataclass
class Message_state:
    messages: list[ModelMessage]

In [15]:
class Main_agent:
    def __init__(self):
        self.agent=Agent(llm, system_prompt="you are a research assistant, you are given a query, leverage what tool(s) to use but do not use the presentation agent unless the user asks for it,\
                          always show the output of the tools, except for the presentation agent",
                          tools=[deep_research_agent,research_editor_tool,quick_research_agent])
        self.deps=Deps( deep_search_results=[], quick_search_results=[])
        self.memory=Message_state(messages=[])

    async def chat(self, query:str):
        result = await self.agent.run(query,deps=self.deps, message_history=self.memory.messages)
        self.memory.messages=result.all_messages()
        return result.data
    
    def reset(self):
        self.memory.messages=[]
        self.deps=Deps( deep_search_results=[], quick_search_results=[])



In [16]:
agent=Main_agent()

In [23]:
await agent.chat('change the image url to a picture of a coral snake')

'I have changed the image URL to a picture of a coral snake. Anything else?\n'

In [12]:
content=agent.deps.deep_search_results[0]


In [24]:
agent.deps.deep_search_results[0]


{'title': '# Snakes in Miami: A Comprehensive Overview',
 'image_url': 'https://www.evergladesholidaypark.com/wp-content/uploads/coral-snake.jpg',
 'paragraphs': [{'title': 'Introduction',
   'content': "Florida is home to a diverse snake population, including both native and non-native species. The Florida Fish and Wildlife Conservation Commission (FWC) plays a crucial role in managing these species, enacting regulations to protect the public and the state's ecosystems. Recent rule changes have added high-risk nonnative reptiles to Florida's Prohibited list, limiting possession to specific purposes. South Florida is home to a large number of snake species, including both native and non-native, with the Burmese python being a major concern due to its devastating impact on native wildlife in the Everglades. Only six of Florida's 44 snake species are venomous, and resources are available to help identify both venomous and non-venomous snakes, as well as to provide guidance on how to hand

In [ ]:
def temp_image(url:str):
    image=requests.get(url)
    image=Image.open(BytesIO(image.content))
    with tempfile.NamedTemporaryFile(mode='w', suffix='.png', delete=False) as temp_file:
        image.save(temp_file.name)
        image_file_path = temp_file.name
    return image_file_path

In [61]:
def paper_to_markdown(paper:dict) -> str:
    """
    Convert a Paper_layout object into a markdown string.
    
    Args:
        paper (Paper_layout): The paper layout to convert
        
    Returns:
        str: The complete markdown document
    """
    
    markdown = []
    
    # Add title
    markdown.append(paper.get('title'))
    markdown.append("\n\n")
    if paper.get('image_url'):
        markdown.append(f"![Image]({paper.get('image_url')})\n")
        markdown.append("\n\n")
    
    # Add body sections
    for section in paper.get('paragraphs'):
        markdown.append(f"## {section.get('title')}\n")
        if section.get('content'):
            markdown.append(section.get('content'))
            markdown.append("\n")


        markdown.append("\n\n")
    
    if paper.get('table'):
        markdown.append(paper.get('table'))
        markdown.append("\n")
    # Add references
    markdown.append("## References\n")
    markdown.append(str(paper.get('references')))
    
    return "".join(markdown)

In [62]:
paper=paper_to_markdown(dict)

In [63]:
print(paper)

Toast POS System for Restaurants: Features and Costs

![Image](<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=752x478 at 0x18506568190>)


## Introduction
Toast POS is a comprehensive, cloud-based restaurant management system tailored for the food service industry, offering a range of features like menu management, order tracking, payment processing, and reporting. Its pricing structure includes subscription-based software plans, hardware options, and payment processing fees, with options for pay-as-you-go or upfront hardware costs affecting processing rates. While Toast offers robust features and specialized tools, particularly for larger or multi-location businesses, its initial hardware and software expenses, along with potential contract complexities and cancellation fees, can be a barrier for some smaller establishments. However, its commitment to customer support, continuous updates, and integrations, solidify its position as a leading POS choice in the restaurant technolo

In [64]:



# Create a temporary file
with tempfile.NamedTemporaryFile(mode='w', suffix='.md', delete=False) as temp_file:
    temp_file.write(paper)
    temp_file_path = temp_file.name

# Create a new document and load from the temporary files
document = Document()
document.LoadFromFile(temp_file_path)

# Save as DOCX
document.SaveToFile("output.docx", FileFormat.Docx)

# Clean up the temporary file
os.unlink(temp_file_path)